In [ ]:
# Part 1:
# given a light board diagram, and buttons
# all lights start off
# buttons simultaneously toggle lights indicated by values in list provided in brackets
# "joltages" provided in curly brackets can be ignored
# find the least number of button presses to obtain the target light output in diagram


def solve_part1(filename):
    data = []
    # parse input
    with open(filename, "r") as f:
        raw = f.read()
        for line in raw.split("\n"):
            arr = line.split(" ")
            diagram = arr[0][1:-1]
            buttons = tuple(tuple(map(int, s[1:-1].split(","))) for s in arr[1:-1])
            joltages = tuple(map(int, arr[-1][1:-1].split(",")))
            data.append({"diagram": diagram, "buttons": buttons, "joltages": joltages})
    result = []
    for item in data:
        diagram = item["diagram"]
        buttons = item["buttons"]
        joltages = item["joltages"]

        # memoization decorator
        memory = {}

        def memoize(func):
            def helper(*args):
                if args not in memory:
                    memory[args] = func(*args)
                return memory[args]

            return helper

        @memoize
        def press_button(board, button):
            new_board = list(board)
            for idx in button:
                new_board[idx] = "#" if new_board[idx] == "." else "."
            return "".join(new_board)

        @memoize
        def find_min_presses(diagram, buttons, board, presses=0):
            if diagram == board:
                return presses
            if presses > 20:
                return float("inf")
            min_presses = float("inf")
            for button in buttons:
                current_board = press_button(board, button)
                current_presses = find_min_presses(
                    diagram, buttons, current_board, presses + 1
                )
                if current_presses < min_presses:
                    min_presses = current_presses
            return min_presses

        min_presses = find_min_presses(diagram, buttons, "." * len(diagram))
        result.append(min_presses)

    return sum(result)


print(f"Part 1 - Test Data: {solve_part1('./data/day10-test.txt')}")
print(f"Part 1 - Actual Data: {solve_part1('./data/day10-data.txt')}")

In [ ]:
# Part 2:
# Now we need to use the buttons to set our joltages to match the provided joltages
# all joltages start at 0
# buttons list counters to increment by 1
# return the sum of the minimum button presses to achieve each joltage in the provided list

import numpy as np
from z3 import Int, Optimize, sat


def solve_part2(filename):
    data = []
    # parse input
    with open(filename, "r") as f:
        raw = f.read()
        for line in raw.split("\n"):
            arr = line.split(" ")
            diagram = arr[0][1:-1]
            buttons = tuple(tuple(map(int, s[1:-1].split(","))) for s in arr[1:-1])
            joltages = tuple(map(int, arr[-1][1:-1].split(",")))
            data.append({"diagram": diagram, "buttons": buttons, "joltages": joltages})
    result = []
    for item in data:
        diagram = item["diagram"]
        buttons = item["buttons"]
        joltages = item["joltages"]

        # I can model the button presses as a system of linear equations
        # my cost fuction is to minimize the number of button presses
        # I can use an optimization library to solve this problem

        a_prep = [[0 for _ in range(len(joltages))] for _ in range(len(buttons))]
        for i, button in enumerate(buttons):
            for j in button:
                a_prep[i][j] = 1  # button i affects joltage j
        a = np.array(a_prep).T
        b = np.array(joltages)

        # :( I saw hinted on reddit that many people used z3 to solve this problem...
        # It feels a bit like cheating... very easy to implement

        # Creating x variables for z3
        x = [Int(f"x{i}") for i in range(len(buttons))]
        opt = Optimize()
        # *** Adding Constraints***
        # adding ax = b constraints
        for i in range(len(joltages)):
            opt.add(sum(a[i][j] * x[j] for j in range(len(buttons))) == b[i])
        # adding x >= 0 constraints
        for i in range(len(buttons)):
            opt.add(x[i] >= 0)
        # *** Adding Objective ***
        opt.minimize(sum(x))
        model = opt.model()
        result.append(sum(model[x[i]].as_long() for i in range(len(buttons))))

    return sum(result)


print(f"Part 2 - Test Data: {solve_part2('./data/day10-test.txt')}")
print(f"Part 2 - Actual Data: {solve_part2('./data/day10-data.txt')}")